In [4]:
using Random
#using Plots
#using Distributions

In [32]:
rand_population_binary(m, n) = [bitrand(n) for i in 1:m] 

rand_population_binary (generic function with 1 method)

In [30]:
function decode_binary(arr)
    bitstring = join(arr)
    return parse(Int, bitstring; base=2)
end

decode_binary (generic function with 1 method)

In [31]:
a = [0, 1, 1]
decode_binary(a)

3

In [25]:
abstract type SelectionMethod end
struct TruncationSelection <: SelectionMethod
    k # top k to keep
end
function select(t::TruncationSelection, y)
    p = sortperm(y)
    return [p[rand(1:t.k, 2)] for i in y]
end

struct TournamentSelection <: SelectionMethod
    k
end
function select(t::TournamentSelection, y)
    getparent() = begin
        p = randperm(length(y))
        p[argmin(y[p[1:t.k]])]
    end
    return [[getparent(), getparent()] for i in y] 
end

struct RouletteWheelSelection <: SelectionMethod end
function select(::RouletteWheelSelection, y)
    y = maximum(y) .- y
    cat = Categorical(normalize(y, 1))
    return [rand(cat, 2) for i in y]
end

select (generic function with 3 methods)

In [26]:
abstract type CrossoverMethod end
struct SinglePointCrossover <: CrossoverMethod end
function crossover(::SinglePointCrossover, a, b)
    i = rand(1:length(a))
    return vcat(a[1:i], b[i+1:end])
end

struct TwoPointCrossover <: CrossoverMethod end
function crossover(::TwoPointCrossover, a, b)
    n = length(a)
    i, j = rand(1:n, 2)
    if i > j
        (i,j) = (j,i)
    end
    return vcat(a[1:i], b[i+1:j], a[j+1:n])
end

struct UniformCrossover <: CrossoverMethod end
function crossover(::UniformCrossover, a, b)
    child = copy(a)
    for i in 1 : length(a)
        if rand() < 0.5
            child[i] = b[i]
        end
    end
    return child
end

crossover (generic function with 3 methods)

In [27]:
abstract type MutationMethod end
struct BitwiseMutation <: MutationMethod
    λ
end
function mutate(M::BitwiseMutation, child)
    return [rand() < M.λ ? !v : v for v in child]
end

struct GaussianMutation <: MutationMethod
    σ
end
function mutate(M::GaussianMutation, child)
    return child + randn(length(child))*M.σ
end

mutate (generic function with 2 methods)

In [33]:
abstract type MutationMethod end
struct GaussianMutation <: MutationMethod
    σ
end

mutate(U::GaussianMutation, child) = child + randn(length(child))*U.σ

function michalewicz(x; m=10)
    return -sum(sin(v)*sin(i*v^2/π)^(2m) for
               (i,v) in enumerate(x))
end

f = x -> michalewicz(x)
xdomain = (0,4)
ydomain = (0,4)

S = TruncationSelection(10)
C = SinglePointCrossover()
M = GaussianMutation(0.1)

m = 40
Random.seed!(0)
population = rand_population_uniform(m, [0.0, 0.0], [4.0,4.0])

function genetic_algorithm(f, population, k_max, S, C, M)
    for k in 1 : k_max
        parents = select(S, f.(population))
        children = [crossover(C,population[p[1]],population[p[2]]) for p in parents]
        population .= mutate.(Ref(M), children)
    end
    population[argmin(f.(population))]
end

genetic_algorithm (generic function with 1 method)